# Ingesting Data into Qdrant
This notebook showcases how to use the `gef-ml` package to create the initial vector database with documents from `/data`.

This assumes data has already been downloaded into `/data/dumps/`

In [ ]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(
    location="http://localhost:6333",
)
vector_store = QdrantVectorStore(client=qdrant_client, collection_name="temp_collection")

In [ ]:
from gef_ml.ingestion import get_pipeline

ingest_pipeline = get_pipeline(vector_store)

## Run the pipeline on a single document

In [ ]:
from llama_index.readers.file import PDFReader

# Testing

In [1]:
from gef_ml.ingestion import StreamingIngestion
from gef_ml.utils import get_qdrant_vectorstore
import aiohttp

In [2]:
vector_store = get_qdrant_vectorstore(collection_name="temp")

ingest_manager = StreamingIngestion(
    directory="../data/to_ingest/", vector_store=vector_store
)

In [3]:
nodes = ingest_manager._ingest_project_id("6930")

/home/sawyer/git/gef-ml/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Parsing nodes: 100%|██████████| 15/15 [00:00<00:00, 1290.03it/s]


In [4]:
from collections.abc import Collection
from typing import Any

def print_types_simplified_corrected(obj, indent=0):
    """
    Recursively prints the types of any object, including nested structures such as dictionaries,
    lists, sets, and other collections, simplifying the output for containers of primitives
    by not listing individual primitives.

    Args:
    - obj: The object whose type information is to be printed.
    - indent (int): The current indentation level for pretty printing.
    """
    prefix = " " * indent
    if isinstance(obj, dict):
        print(f"{prefix}{type(obj).__name__} containing:")
        for key, value in obj.items():
            print(f"{prefix}  Key type: {type(key).__name__}, Value type: ", end="")
            print_types_simplified_corrected(value, indent + 4)
    elif isinstance(obj, Collection) and not isinstance(obj, str):
        item_types = {type(item).__name__ for item in obj}
        if len(item_types) == 1:
            item_type = next(iter(item_types))  # Use next(iter()) to safely access the first item
            if item_type in ["int", "float", "str", "bool", "NoneType"]:
                print(f"{prefix}{type(obj).__name__} of {item_type} items")
                return
        print(f"{prefix}{type(obj).__name__} of mixed types")
        for item in obj:
            print_types_simplified_corrected(item, indent + 4)
    else:
        print(f"{prefix}{type(obj).__name__}")

In [5]:
def print_types_with_key_names(obj, key_name=None, indent=0):
    """
    Recursively prints the types of any object, including nested structures such as dictionaries,
    lists, sets, and other collections, simplifying the output for containers of primitives
    by not listing individual primitives. It includes the key names for dictionary entries.

    Args:
    - obj: The object whose type information is to be printed.
    - key_name: The name of the key (if any) associated with the current object.
    - indent (int): The current indentation level for pretty printing.
    """
    prefix = " " * indent
    key_str = f"'{key_name}' " if key_name else ""
    if isinstance(obj, dict):
        print(f"{prefix}{key_str}{type(obj).__name__} containing:")
        for key, value in obj.items():
            print_types_with_key_names(value, key_name=key, indent=indent + 4)
    elif isinstance(obj, Collection) and not isinstance(obj, str):
        item_types = {type(item).__name__ for item in obj}
        if len(item_types) == 1:
            item_type = next(iter(item_types))  # Use next(iter()) to safely access the first item
            if item_type in ["int", "float", "str", "bool", "NoneType"]:
                print(f"{prefix}{key_str}{type(obj).__name__} of {item_type} items")
                return
        print(f"{prefix}{key_str}{type(obj).__name__} of mixed types")
        for item in obj:
            print_types_with_key_names(item, indent=indent + 4)
    else:
        print(f"{prefix}{key_str}{type(obj).__name__}")

In [6]:
session = aiohttp.ClientSession()

In [7]:
embedding = ingest_manager.fetch_embedding(session=session, node=nodes[0], model="togethercomputer/m2-bert-80M-32k-retrieval")

In [8]:
response = await embedding
response

[-0.055081844,
 -0.08664002,
 0.020069068,
 -0.026689788,
 0.0037623271,
 0.004335452,
 0.041014127,
 -0.02275657,
 -0.07045815,
 -0.057266604,
 0.022192955,
 0.005135613,
 -0.039583143,
 0.0022596351,
 -0.005641797,
 0.07242038,
 0.005358643,
 -0.009924466,
 -0.118348025,
 -0.0045331237,
 -0.0031099517,
 0.0036700212,
 0.022573883,
 -0.04360209,
 0.09557577,
 0.008802387,
 -0.07435157,
 0.02138878,
 0.07159196,
 -0.0136822285,
 0.02585229,
 0.01291477,
 -0.0012913516,
 -0.008894813,
 -0.088698424,
 -0.05153556,
 -0.013578927,
 0.043187957,
 0.024779335,
 0.04099302,
 0.04088058,
 -0.0039694263,
 -0.022314252,
 -0.056680772,
 -0.034085635,
 -0.0045354506,
 0.012546801,
 0.034222495,
 0.012105263,
 -0.04566977,
 -0.039607733,
 -0.028408382,
 0.02408912,
 -0.023830382,
 0.006820996,
 0.064021744,
 0.012930531,
 0.025703147,
 -0.07236337,
 -0.06799486,
 -0.022363372,
 0.056123395,
 0.018476278,
 -0.025521047,
 0.024988173,
 -0.033039622,
 -0.013058345,
 0.05830469,
 0.013880926,
 -0.03553

In [9]:
print_types_with_key_names(response)

list of float items


In [ ]:
response['data'][0]['embedding']

In [9]:
embeddings = ingest_manager.generate_embeddings_rest(nodes)

In [10]:
resp = await embeddings

Generating embeddings: 100%|██████████| 17/17 [00:08<00:00,  2.03it/s]


In [11]:
print_types_with_key_names(resp)

list of mixed types
    TextNode
    TextNode
    TextNode
    TextNode
    TextNode
    TextNode
    TextNode
    TextNode
    TextNode
    TextNode
    TextNode
    TextNode
    TextNode
    TextNode
    TextNode
    TextNode
    TextNode


In [12]:
resp

[TextNode(id_='cf2758e6-d5f4-42f9-b5e2-524a62be9012', embedding=[-0.055124816, -0.08664272, 0.020074755, -0.026682796, 0.0037684033, 0.004358757, 0.04100559, -0.022756755, -0.070461735, -0.057259854, 0.02218899, 0.0051160646, -0.03957953, 0.0022795012, -0.005645436, 0.072412975, 0.0053681345, -0.009910253, -0.1183348, -0.004518301, -0.0030965277, 0.0036562504, 0.022590628, -0.04359483, 0.09557422, 0.0088075055, -0.074348204, 0.021379469, 0.07158423, -0.013692413, 0.0258643, 0.012917043, -0.0012985889, -0.008883312, -0.0886897, -0.05154296, -0.01358302, 0.043190833, 0.024770929, 0.04100997, 0.040879045, -0.003985929, -0.02230572, -0.056698374, -0.03408884, -0.004530654, 0.01254666, 0.034234498, 0.012120526, -0.045673244, -0.039607506, -0.028410994, 0.024080127, -0.023822892, 0.006795661, 0.064034015, 0.012940067, 0.02568392, -0.07239142, -0.06798013, -0.022364227, 0.056112364, 0.018463342, -0.025536722, 0.025003772, -0.033047292, -0.013074572, 0.05829042, 0.013872085, -0.035520088, -0.0

In [15]:
from llama_index.core.schema import MetadataMode
nodes[0].get_content(metadata_mode=MetadataMode.EMBED)

'page_label: 1\nfilename: p6930_doc2.pdf\nextension: .pdf\nproject_id: 6930\ndoc_id: 2\n\n_______________________________________________________________________________________ _____________________________  \n \nGEF ID:  6930  \nCountry/Region:  China  \nProject Title:  Energy Efficiency Improvement in Public Sector Buildings   \nGEF Agency:  UNDP  GEF Agency Project ID:  5395 (UNDP)  \nType of Trust Fund:  GEF Trust Fund  GEF Focal Area (s):  Climate Change  \nGEF -5 Focal Area/ LDCF/SCCF Objective (s):   \nAnticipated Financing  PPG:  $200,000  Project Grant:  $8,932,420  \nCo-financing:  $62,500,000  Total Project Cost:  $71,632,420  \nPIF Approval:   Council Approval/Expected:  October 01, 2015  \nCEO Endorsement/Approval   Expected Project Start Date:   \nProgram Manager:  Ming Yang  Agency Contact Person:  Manuel L. Soriano  \n \nReview Criteria  Questions  Secretariat Comment at PIF (PFD)/Work \nProgram Inclusion 1 Secretariat Comment At CEO \nEndorsement(FSP)/Approval (MSP)  

In [23]:
nodes[0].get_content(metadata_mode=MetadataMode.LLM)

'page_label: 1\nfilename: p6930_doc2.pdf\nextension: .pdf\nproject_id: 6930\ndoc_id: 2\n\n_______________________________________________________________________________________ _____________________________  \n \nGEF ID:  6930  \nCountry/Region:  China  \nProject Title:  Energy Efficiency Improvement in Public Sector Buildings   \nGEF Agency:  UNDP  GEF Agency Project ID:  5395 (UNDP)  \nType of Trust Fund:  GEF Trust Fund  GEF Focal Area (s):  Climate Change  \nGEF -5 Focal Area/ LDCF/SCCF Objective (s):   \nAnticipated Financing  PPG:  $200,000  Project Grant:  $8,932,420  \nCo-financing:  $62,500,000  Total Project Cost:  $71,632,420  \nPIF Approval:   Council Approval/Expected:  October 01, 2015  \nCEO Endorsement/Approval   Expected Project Start Date:   \nProgram Manager:  Ming Yang  Agency Contact Person:  Manuel L. Soriano  \n \nReview Criteria  Questions  Secretariat Comment at PIF (PFD)/Work \nProgram Inclusion 1 Secretariat Comment At CEO \nEndorsement(FSP)/Approval (MSP)  